**классификатор текстов на естественном языке**


Евгений Борисов borisov.e@solarl.ru

In [1]:
# Sebastian Raschka   Python Machine Learning  - Packt Publishing Ltd, 2015

In [2]:
import numpy as np
import pandas as pd
# import pickle
# import sqlite3
import re

In [ ]:
from Stemmer import Stemmer
# sudo pacman -S python-pystemmer

#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import HashingVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
pd.options.display.max_colwidth = 200  

---

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))
data.sample(2)

записей: 3196


,text,tag
791,"Войтюшкевичу отказали в площадке для проведения разрешенного концерта в Минске\n\n7 декабря 2016 в 20:22\n\nTUT.BY\n\nКонцерт группы WZ-orkiestra и Дмитрия Войтюшкевича отменен после того, как его...",culture
547,"На трассе М6 авария с тяжелыми последствиями: VW Golf въехал в поворачивающий МАЗ\n\n5 декабря 2016 в 9:43\n\nAUTO.TUT.BY\n\nВ воскресенье, 4 декабря, на трассе М6 VW Golf въехал в МАЗ. Легковушка...",incident


In [5]:
# классы
print(len(set(data['tag'])))
set(data['tag'])

13


{'auto',
 'culture',
 'economics',
 'health',
 'incident',
 'politics',
 'realty',
 'reclama',
 'science',
 'social',
 'sport',
 'tech',
 'woman'}

---

In [36]:
from Stemmer import Stemmer

# очистка текста с помощью regexp
def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    text = re.sub( r'https?://[\S]+', ' url ', text) # замена интернет ссылок
    text = re.sub( r'[\w\./]+\.[a-z]+', ' url ', text) 
 
    # text = re.sub( r'\d+[-/\.]\d+[-/\.]\d+', ' date ', text) # замена даты и времени
    # text = re.sub( r'\d+ ?гг?', ' date ', text) 
    # text = re.sub( r'\d+:\d+(:\d+)?', ' time ', text) 
    # text = re.sub( r'@\w+', ' tname ', text ) # замена имён twiter
    # text = re.sub( r'#\w+', ' htag ', text ) # замена хештегов

    text = re.sub( r'<[^>]*>', ' ', text) # удаление html тагов
    text = re.sub( r'[\W]+', ' ', text ) # удаление лишних символов

    stemmer = Stemmer('russian')
    text = ' '.join( stemmer.stemWords( text.split() ) ) 

#     stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
#            'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
#     remove = r'\b('+'|'.join(stw)+')\b'
#     text = re.sub(remove,' ', text)
    
    text = re.sub( r'\b\w\b', ' ', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text 

---

In [37]:
data['text_clean'] = data['text'].apply(text_cleaner)

In [43]:
# data

In [39]:
# docs = [
#     'The sun is shining',
#     'The weather is sweet',
#     'The sun is shining and the weather is sweet',
#   ]

# docs = data['text'].values
docs = data['text_clean'].values

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
bag = count.fit_transform(docs)

In [41]:
print( 'словарь:', len(count.vocabulary_), 'слов\n')  

словарь: 35246 слов



In [44]:
# print(count.vocabulary_)
vcb = sorted(list( count.vocabulary_.keys() ))
# vcb[5500:5550]

In [45]:
# print( bag.toarray() )

In [ ]:
# np.set_printoptions(precision=2)

In [ ]:
# tf-idf
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2',smooth_idf=True)

tfidf.fit_transform(count.fit_transform(docs)).toarray()


---

In [50]:
def tokenizer(text):  return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [49]:
# The following code shows how to use the Porter stemming algorithm:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text): return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')
# ['runner', 'like', 'run', 'and', 'thu', 'they', 'run']


['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [51]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mechanoid/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [53]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [55]:
from nltk.corpus import stopwords
stop = stopwords.words('russian')
[w for w in tokenizer_porter('отказали в площадке для проведения разрешенного концерта')[-10:] if w not in stop]

['отказали', 'площадке', 'проведения', 'разрешенного', 'концерта']

In [ ]:
# from sklearn.feature_extraction.text import HashingVectorizer
# from sklearn.linear_model import SGDClassifier
# vect = HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=tokenizer)
# clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
# doc_stream = stream_docs(path='movie_data.csv')

---

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

pipe_txt = Pipeline([
               ('hashvect', HashingVectorizer() ),
               ('tfidf', TfidfTransformer(use_idf=False )),
               ])

pipe_txt = pipe_txt.fit(data['text'])
pipe_txt.transform(data['text'])

# hashvect = HashingVectorizer()
# tfidf = TfidfTransformer(use_idf=False )

# hashvect = hashvect.fit(data['text'])
# # hashvect.transform(data['text']).todense()
# tfidf = tfidf.fit(  hashvect.transform(data['text']) )
# tfidf.transform( hashvect.transform(data['text']) ).todense()



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



# text_clf = Pipeline([
#                ('covect', CountVectorizer() ),
#                ('tfidf', TfidfTransformer(preprocessor=text_cleaner,use_idf=False )),
#                ('clf', SGDClassifier(loss='hinge')),
#                ])




# covect = CountVectorizer()
# tfidf = TfidfTransformer(preprocessor=text_cleaner,use_idf=False )




In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer()

In [ ]:
# print("[i] обучение классификатора...")

# text_clf = Pipeline([
#                ('hashvect', HashingVectorizer() ),
#                ('tfidf', TfidfTransformer(use_idf=False )),
#                ('clf', SGDClassifier(loss='hinge')),
#                ])

# text_clf = Pipeline([
#                ('covect', CountVectorizer() ),
#                ('tfidf', TfidfTransformer(preprocessor=text_cleaner,use_idf=False )),
#                ('clf', SGDClassifier(loss='hinge')),
#                ])

# text_clf = Pipeline([
#                 ('tfidf', TfidfVectorizer()),
#                 ('clf', SGDClassifier(loss='hinge')),
#                 ])

# text_clf.fit(D['train']['x'], D['train']['y'])

In [ ]:
print("[i] тестируем...")

predicted = text_clf.predict( D['train']['x'] )
print("\taccuracy train: ", accuracy_score(  D['train']['y'] , predicted) )

predicted = text_clf.predict( D['test']['x'] )
print("\taccuracy test: ", accuracy_score(  D['test']['y'] , predicted) )

---

In [ ]:
# # разделение набора текстов (data)
# # на тестовый и учебный наборы
# # случайным образом в заданной пропорции (validation_split)
# #
# def train_test_split( data, validation_split = 0.2):
#     sz = len(data['text'])
#     indices = np.arange(sz)
#     np.random.shuffle(indices)

#     X = [ data['text'][i] for i in indices ]
#     Y = [ data['tag'][i] for i in indices ]
#     nb_validation_samples = int( validation_split * sz )

#     return { 
#         'train': { 'x': X[:-nb_validation_samples], 'y': Y[:-nb_validation_samples]  },
#         'test': { 'x': X[-nb_validation_samples:], 'y': Y[-nb_validation_samples:]  }
#     }

In [ ]:
# print("[i] загружаем данные...")
# data = load_data()
# print("\tсчитано: ",len(data['text']))

In [ ]:
#print("[i] очистка данных...")
#data['text'] = [ text_cleaner(t) for t in data['text'] ]

# print("[i] разделение данных...")
# D = train_test_split( data )